In [1]:
!pip install torch transformers datasets peft accelerate bitsandbytes scikit-learn tqdm trl # Tejaaa yeh bhi daalna tha lol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 28.7 MB/s eta 0:00:00


In [2]:
!unzip models.zip

Archive:  models.zip
   creating: models/
   creating: models/gemma3-270m-tool-calling/
   creating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/README.md  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/adapter_model.safetensors  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/special_tokens_map.json  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/adapter_config.json  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/chat_template.jinja  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/tokenizer.json  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/tokenizer_config.json  
 extracting: models/gemma3-270m-tool-calling/checkpoint-epoch-1/added_tokens.json  
  inflating: models/gemma3-270m-tool-calling/checkpoint-epoch-1/tokenizer.model  
   creating: models/gemma3-270m-tool-calling-final/
  inflating: models/gemma3-270

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import json
import random
from datasets import load_dataset, Dataset, load_from_disk
from tqdm import tqdm


def create_tool_calling_prompt(query, tools):
    """
    Creates a formatted prompt for tool calling following Gemma-3 format.
    """
    # Create tool definitions
    tool_defs = []
    for tool in tools:
        func_name = tool.get('name')
        params = tool.get('parameters', {}).get('properties', {})
        param_defs = [f"{pname}: {p.get('type') if isinstance(p, dict) else p}"
                      for pname, p in params.items()]
        tool_defs.append(f"def {func_name}({', '.join(param_defs)}):")

    tool_defs_str = "\n".join(tool_defs)

    # Build the prompt
    prompt = (
        f"<start_of_turn>user\n"
        f"You are an expert AI assistant with access to a suite of tools. Use them to answer the user's question.\n"
        f"Tools:\n{tool_defs_str}\n"
        f"{query}<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )

    return prompt


def create_chosen_response(answers):
    """
    Creates the correct (chosen) tool calling response.
    """
    tool_calls = []
    for answer in answers:
        func_name = answer.get('name')
        args = answer.get('arguments', {})
        tool_calls.append(f"{func_name}(**{json.dumps(args)})")

    tool_calls_str = "\n".join(tool_calls)
    response = f"<tool_code>\n{tool_calls_str}<end_of_turn>"

    return response


def create_rejected_response(answers, tools):
    """
    Creates an incorrect (rejected) tool calling response.
    This simulates common errors:
    1. Wrong function name
    2. Missing required arguments
    3. Wrong argument values
    4. Hallucinated functions
    5. Wrong number of function calls
    """
    rejection_type = random.choice([
        'wrong_function',
        'missing_args',
        'wrong_args',
        'hallucinated',
        'extra_calls',
        'no_tool_use'
    ])

    tool_calls = []

    if rejection_type == 'wrong_function':
        # Use wrong function name
        for answer in answers:
            # Pick a random different function from tools
            available_funcs = [t['name'] for t in tools if t['name'] != answer.get('name')]
            if available_funcs:
                wrong_func = random.choice(available_funcs)
                args = answer.get('arguments', {})
                tool_calls.append(f"{wrong_func}(**{json.dumps(args)})")
            else:
                # Fallback: use original
                func_name = answer.get('name')
                args = answer.get('arguments', {})
                tool_calls.append(f"{func_name}(**{json.dumps(args)})")

    elif rejection_type == 'missing_args':
        # Remove some required arguments
        for answer in answers:
            func_name = answer.get('name')
            args = answer.get('arguments', {})
            if args:
                # Remove random argument
                args_copy = args.copy()
                if len(args_copy) > 1:
                    key_to_remove = random.choice(list(args_copy.keys()))
                    del args_copy[key_to_remove]
                tool_calls.append(f"{func_name}(**{json.dumps(args_copy)})")
            else:
                tool_calls.append(f"{func_name}()")

    elif rejection_type == 'wrong_args':
        # Use wrong argument values
        for answer in answers:
            func_name = answer.get('name')
            args = answer.get('arguments', {})
            wrong_args = {}
            for key, value in args.items():
                if isinstance(value, str):
                    wrong_args[key] = "WRONG_VALUE"
                elif isinstance(value, int):
                    wrong_args[key] = 99999
                elif isinstance(value, list):
                    wrong_args[key] = []
                else:
                    wrong_args[key] = None
            tool_calls.append(f"{func_name}(**{json.dumps(wrong_args)})")

    elif rejection_type == 'hallucinated':
        # Call non-existent function
        tool_calls.append(f"hallucinated_function(**{json.dumps({'param': 'value'})})")

    elif rejection_type == 'extra_calls':
        # Add extra unnecessary calls
        for answer in answers:
            func_name = answer.get('name')
            args = answer.get('arguments', {})
            tool_calls.append(f"{func_name}(**{json.dumps(args)})")
        # Add extra call
        if tools:
            extra_func = random.choice(tools)['name']
            tool_calls.append(f"{extra_func}(**{json.dumps({})})")

    elif rejection_type == 'no_tool_use':
        # Just give a text response without using tools
        return "I'll help you with that. Let me think about it.<end_of_turn>"

    tool_calls_str = "\n".join(tool_calls)
    response = f"<tool_code>\n{tool_calls_str}<end_of_turn>"

    return response


def prepare_dpo_dataset(dataset_name="Salesforce/xlam-function-calling-60k",
                       split="train",
                       num_samples=5000,
                       output_path="./data/dpo_tool_calling"):
    """
    Prepares a DPO dataset from the tool calling dataset.

    DPO requires:
    - prompt: The input prompt
    - chosen: The preferred/correct response
    - rejected: The non-preferred/incorrect response
    """
    print(f"Loading dataset from {dataset_name}...")
    dataset = load_dataset(dataset_name, split=split)

    # Use subset for DPO (you can adjust this)
    if num_samples and num_samples < len(dataset):
        dataset = dataset.select(range(num_samples))

    print(f"Processing {len(dataset)} examples for DPO...")

    dpo_data = {
        'prompt': [],
        'chosen': [],
        'rejected': []
    }

    for example in tqdm(dataset, desc="Creating preference pairs"):
        try:
            query = example['query']
            tools = json.loads(example['tools'])
            answers = json.loads(example['answers'])

            # Create prompt
            prompt = create_tool_calling_prompt(query, tools)

            # Create chosen (correct) response
            chosen = create_chosen_response(answers)

            # Create rejected (incorrect) response
            rejected = create_rejected_response(answers, tools)

            dpo_data['prompt'].append(prompt)
            dpo_data['chosen'].append(chosen)
            dpo_data['rejected'].append(rejected)

        except Exception as e:
            print(f"Error processing example: {e}")
            continue

    # Create HuggingFace Dataset
    dpo_dataset = Dataset.from_dict(dpo_data)

    print(f"\nDPO Dataset created with {len(dpo_dataset)} examples")
    print(f"Sample example:")
    print(f"Prompt: {dpo_dataset[0]['prompt'][:200]}...")
    print(f"Chosen: {dpo_dataset[0]['chosen'][:200]}...")
    print(f"Rejected: {dpo_dataset[0]['rejected'][:200]}...")

    # Save dataset
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    dpo_dataset.save_to_disk(output_path)
    print(f"\nDPO dataset saved to {output_path}")

    return dpo_dataset


def validate_dpo_dataset(dataset_path="./data/dpo_tool_calling"):
    """
    Validates that the DPO dataset has the correct format.
    """
    print(f"Loading DPO dataset from {dataset_path}...")
    dataset = load_from_disk(dataset_path)

    print(f"\nDataset info:")
    print(f"Number of examples: {len(dataset)}")
    print(f"Columns: {dataset.column_names}")

    # Check required columns
    required_cols = ['prompt', 'chosen', 'rejected']
    for col in required_cols:
        if col not in dataset.column_names:
            print(f"ERROR: Missing required column '{col}'")
            return False

    # Show a few examples
    print(f"\nExample 1:")
    print(f"Prompt length: {len(dataset[0]['prompt'])} chars")
    print(f"Chosen length: {len(dataset[0]['chosen'])} chars")
    print(f"Rejected length: {len(dataset[0]['rejected'])} chars")
    print(f"\nPrompt preview:\n{dataset[0]['prompt'][:300]}...")
    print(f"\nChosen preview:\n{dataset[0]['chosen'][:300]}...")
    print(f"\nRejected preview:\n{dataset[0]['rejected'][:300]}...")

    print("\n✓ DPO dataset validation passed!")
    return True


def main():
    """
    Main function - can be called directly in notebooks or from command line.
    """
    import sys

    # Check if running in Jupyter/Colab (will have -f argument)
    is_notebook = any('jupyter' in arg or 'ipykernel' in arg for arg in sys.argv)

    if is_notebook or 'ipykernel_launcher' in sys.argv[0] or 'colab_kernel_launcher' in sys.argv[0]:
        # Running in notebook - use default parameters
        print("Running in notebook mode with default parameters...")
        print("To customize, call prepare_dpo_dataset() directly with your parameters")

        dpo_dataset = prepare_dpo_dataset(
            dataset_name="Salesforce/xlam-function-calling-60k",
            split="train",
            num_samples=6000,
            output_path="./data/dpo_tool_calling"
        )

        # Validate it
        print("\n" + "="*50)
        validate_dpo_dataset("./data/dpo_tool_calling")

    else:
        # Running from command line - use argparse
        import argparse

        parser = argparse.ArgumentParser(description="Prepare DPO dataset for tool calling")
        parser.add_argument('--dataset', type=str, default="Salesforce/xlam-function-calling-60k",
                           help='Source dataset name')
        parser.add_argument('--split', type=str, default="train",
                           help='Dataset split to use')
        parser.add_argument('--num_samples', type=int, default=5000,
                           help='Number of samples to use (None for all)')
        parser.add_argument('--output', type=str, default="./data/dpo_tool_calling",
                           help='Output path for DPO dataset')
        parser.add_argument('--validate', action='store_true',
                           help='Only validate existing dataset')

        args = parser.parse_args()

        if args.validate:
            validate_dpo_dataset(args.output)
        else:
            # Prepare the dataset
            dpo_dataset = prepare_dpo_dataset(
                dataset_name=args.dataset,
                split=args.split,
                num_samples=args.num_samples,
                output_path=args.output
            )

            # Validate it
            print("\n" + "="*50)
            validate_dpo_dataset(args.output)


if __name__ == "__main__":
    main()

Running in notebook mode with default parameters...
To customize, call prepare_dpo_dataset() directly with your parameters
Loading dataset from Salesforce/xlam-function-calling-60k...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

xlam_function_calling_60k.json:   0%|          | 0.00/96.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Processing 6000 examples for DPO...


Creating preference pairs: 100%|██████████| 6000/6000 [00:00<00:00, 8051.86it/s]


DPO Dataset created with 6000 examples
Sample example:
Prompt: <start_of_turn>user
You are an expert AI assistant with access to a suite of tools. Use them to answer the user's question.
Tools:
def live_giveaways_by_type():
Where can I find live giveaways for bet...
Chosen: <tool_code>
live_giveaways_by_type(**{"type": "beta"})
live_giveaways_by_type(**{"type": "game"})<end_of_turn>...
Rejected: <tool_code>
live_giveaways_by_type(**{"type": "beta"})
live_giveaways_by_type(**{"type": "game"})<end_of_turn>...


Saving the dataset (0/1 shards):   0%|          | 0/6000 [00:00<?, ? examples/s]


DPO dataset saved to ./data/dpo_tool_calling

Loading DPO dataset from ./data/dpo_tool_calling...

Dataset info:
Number of examples: 6000
Columns: ['prompt', 'chosen', 'rejected']

Example 1:
Prompt length: 254 chars
Chosen length: 110 chars
Rejected length: 110 chars

Prompt preview:
<start_of_turn>user
You are an expert AI assistant with access to a suite of tools. Use them to answer the user's question.
Tools:
def live_giveaways_by_type():
Where can I find live giveaways for beta access and games?<end_of_turn>
<start_of_turn>model
...

Chosen preview:
<tool_code>
live_giveaways_by_type(**{"type": "beta"})
live_giveaways_by_type(**{"type": "game"})<end_of_turn>...

Rejected preview:
<tool_code>
live_giveaways_by_type(**{"type": "beta"})
live_giveaways_by_type(**{"type": "game"})<end_of_turn>...

✓ DPO dataset validation passed!


In [5]:
import os
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model
from trl import DPOConfig, DPOTrainer


def main():
    # Set CUDA device (User can override)
    if "CUDA_VISIBLE_DEVICES" not in os.environ:
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    model_name = "google/gemma-3-270m-it"
    # Path to the SFT model (trained with your previous script)
    sft_adapter_path = "./models/gemma3-270m-tool-calling-final"
    output_dir = "./models/gemma3-270m-tool-calling-dpo"

    print(f"Loading DPO dataset...")
    try:
        dataset = load_from_disk("./data/dpo_tool_calling")
    except Exception:
        print("Please run src/prepare_dpo_data.py first!")
        return

    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = 'left'

    print("Loading SFT Model (Base + Adapter)...")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        trust_remote_code=True,
        device_map="auto"
    )

    # Load the SFT adapter
    try:
        model = PeftModel.from_pretrained(base_model, sft_adapter_path)
        # For DPO with LoRA, we generally want to train on top of the SFT policy.
        # With 4bit, we cannot merge easily.
        # TRL's DPOTrainer handles PEFT models automatically if we pass peft_config.
        # The SFT model serves as the initial policy, and the frozen base serves as reference.
        print("SFT Adapter loaded successfully.")
    except Exception as e:
        print(f"Could not load SFT adapter from {sft_adapter_path}. Error: {e}")
        print("Using base model only (Not recommended for DPO)")
        model = base_model

    # LoRA configuration for DPO
    # We add new LoRA adapters on top for further fine-tuning
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # DPO Training configuration
    training_args = DPOConfig(
        output_dir=output_dir,
        beta=0.1,  # Strength of KL penalty (same as Phi-3 example)
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=5e-5,  # Lower LR for DPO (same as Phi-3 example)
        logging_steps=10,
        save_steps=100,
        num_train_epochs=1,
        max_length=1024,
        max_prompt_length=512,
        remove_unused_columns=False,
        bf16=True,
    )

    # Initialize DPO Trainer
    trainer = DPOTrainer(
        model=model,
        ref_model=None,  # None because we use PEFT; base model acts as reference
        args=training_args,
        train_dataset=dataset,
        processing_class=tokenizer,  # Replaces 'tokenizer' in newer TRL versions
        peft_config=peft_config,
    )

    print("Starting DPO training...")
    trainer.train()
    print("DPO Training finished.")

    # Save the final DPO model
    trainer.save_model(f"{output_dir}-final")
    tokenizer.save_pretrained(f"{output_dir}-final")
    print(f"DPO Model saved to {output_dir}-final")


if __name__ == "__main__":
    main()

Loading DPO dataset...
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ishaan300104 (ishaan300104-Indian Institute of Technology, Jodhpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.428500
20,0.184200
30,0.239800
40,0.197400
50,0.152600
60,0.141100
70,0.166600
80,0.119600
90,0.161300
100,0.105200


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn

DPO Training finished.
DPO Model saved to ./models/gemma3-270m-tool-calling-dpo-final


In [9]:
!zip -r /content/gemma.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/gemma3-270m-tool-calling/ (stored 0%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/ (stored 0%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/README.md (deflated 65%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/adapter_model.safetensors (deflated 7%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/special_tokens_map.json (deflated 77%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/adapter_config.json (deflated 58%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/chat_template.jinja (deflated 70%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/tokenizer.json (deflated 83%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/tokenizer_config.json (deflated 97%)
  adding: content/models/gemma3-270m-tool-calling/checkpoint-epoch-1/added_tokens.json (store

In [ ]:
!zip -r /content/data_dpo.zip /content/data

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import json
from tqdm import tqdm
import re
import gc
import os
from datasets import load_dataset, Dataset
import logging
from datetime import datetime
import numpy as np

# --- CONFIGURATION & LOGGING ---
RESULTS_FILE = "dpo_evaluation_results.json"
LOG_FILE = f"dpo_eval_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(LOG_FILE),
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

# --- HELPER FUNCTIONS ---

def load_tool_calling_dataset(dataset_name="Salesforce/xlam-function-calling-60k", split="train", start=None, end=None) -> Dataset:
    try:
        dataset = load_dataset(dataset_name, split=split)
        if start is not None and end is not None:
            dataset = dataset.select(range(start, end))
        return dataset
    except Exception as e:
        logger.error(f"Error loading dataset: {e}")
        return None

def format_prompt_for_eval(sample):
    query = sample['query']
    tools = json.loads(sample['tools']) if isinstance(sample['tools'], str) else sample['tools']

    tool_defs = []
    for tool in tools:
        func_name = tool.get('name')
        params = tool.get('parameters', {}).get('properties', {})
        param_defs = [f"{pname}: {p.get('type') if isinstance(p, dict) else p}" for pname, p in params.items()]
        tool_defs.append(f"def {func_name}({', '.join(param_defs)}):")

    tool_defs_str = "\n".join(tool_defs)

    prompt = (
        f"<start_of_turn>user\n"
        f"You are an expert AI assistant with access to a suite of tools. Use them to answer the user's question.\n"
        f"Tools:\n{tool_defs_str}\n"
        f"{query}<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )
    return prompt

def parse_tool_calls(prediction_str: str):
    prediction_str = prediction_str.split('<end_of_turn>')[0].strip()
    if "<tool_code>" in prediction_str:
        prediction_str = prediction_str.split("<tool_code>")[-1].strip()

    pattern = re.compile(r"(\w+)\((.*?)\)")
    matches = pattern.findall(prediction_str)

    calls = []
    for match in matches:
        func_name = match[0]
        args_str = match[1].strip()

        if func_name in ['print', 'return', 'exit', 'assert']:
            continue

        try:
            if args_str.startswith('**'):
                args_str = args_str[2:].strip()

            args_dict = json.loads(args_str)
            calls.append({"name": func_name, "arguments": args_dict})

        except Exception:
            logger.warning(f"Failed to parse arguments for call: {func_name}({args_str}) - Skipping.")
            continue

    return calls

def calculate_metrics(predictions, ground_truths):
    gt_list = json.loads(ground_truths) if isinstance(ground_truths, str) else ground_truths

    pred_set = {json.dumps(p, sort_keys=True) for p in predictions}
    gt_set = {json.dumps(g, sort_keys=True) for g in gt_list}

    correct_calls = len(pred_set.intersection(gt_set))

    precision = correct_calls / len(pred_set) if len(pred_set) > 0 else 0
    recall = correct_calls / len(gt_set) if len(gt_set) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    exact_match = 1 if pred_set == gt_set and len(gt_set) > 0 else 0

    return precision, recall, f1, exact_match

def calculate_tool_select_metrics(predictions, ground_truths):
    """
    Calculates Tool Select F1 score based only on the set of function names used.
    """
    gt_list = json.loads(ground_truths) if isinstance(ground_truths, str) else ground_truths

    pred_tools = {p['name'] for p in predictions}
    gt_tools = {g['name'] for g in gt_list}

    correct_tool_selections = len(pred_tools.intersection(gt_tools))

    ts_precision = correct_tool_selections / len(pred_tools) if len(pred_tools) > 0 else 0
    ts_recall = correct_tool_selections / len(gt_tools) if len(gt_tools) > 0 else 0

    ts_f1 = 2 * (ts_precision * ts_recall) / (ts_precision + ts_recall) if (ts_precision + ts_recall) > 0 else 0

    return ts_f1

def calculate_hallucination_rate(predictions, available_tools):
    if not predictions:
        return 0

    available_tools_list = json.loads(available_tools) if isinstance(available_tools, str) else available_tools

    available_tool_map = {
        tool['name']: set(tool.get('parameters', {}).get('properties', {}).keys())
        for tool in available_tools_list
    }

    hallucinated_calls_count = 0

    for pred in predictions:
        if pred['name'] not in available_tool_map:
            hallucinated_calls_count += 1
            continue

        valid_args = available_tool_map[pred['name']]
        for arg_name in pred['arguments']:
            if arg_name not in valid_args:
                hallucinated_calls_count += 1
                break

    return hallucinated_calls_count / len(predictions)

def save_metrics_to_json(raw_metrics, total_samples, file_path=RESULTS_FILE):
    """Aggregates and saves all evaluation results to a JSON file."""

    final_results = {}

    for model_name, res_list in raw_metrics.items():
        num_valid_samples = len(res_list)

        if num_valid_samples == 0:
            agg_data = {
                "total_samples": total_samples,
                "valid_samples": 0,
                "error": "No valid predictions processed."
            }
        else:
            p_sum = sum(res['p'] for res in res_list)
            r_sum = sum(res['r'] for res in res_list)
            f1_sum = sum(res['f1'] for res in res_list)
            em_sum = sum(res['em'] for res in res_list)
            hr_sum = sum(res['hr'] for res in res_list)
            ts_f1_sum = sum(res['ts_f1'] for res in res_list)

            agg_data = {
                "total_samples": total_samples,
                "valid_samples": num_valid_samples,
                "avg_precision": p_sum / num_valid_samples,
                "avg_recall": r_sum / num_valid_samples,
                "avg_f1_score": f1_sum / num_valid_samples,
                "avg_tool_select_f1_score": ts_f1_sum / num_valid_samples,
                "exact_match_rate": em_sum / num_valid_samples,
                "avg_hallucination_rate": hr_sum / num_valid_samples,
                "individual_samples": res_list
            }

        final_results[model_name] = agg_data

    with open(file_path, 'w') as f:
        json.dump(final_results, f, indent=4)

    logger.info(f"All evaluation metrics saved to {file_path}")
    return final_results

# --- Main Evaluation Logic ---
def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    logger.info("Starting DPO evaluation script...")

    base_model_name = "google/gemma-3-270m-it"
    sft_model_path = "./models/gemma3-270m-tool-calling-final"
    dpo_model_path = "./models/gemma3-270m-tool-calling-dpo-final"

    # Load evaluation subset (use different range than training: 7000-8000)
    eval_dataset = load_tool_calling_dataset(split="train", start=7000, end=8000)
    if not eval_dataset:
        logger.error("Failed to load evaluation dataset. Exiting.")
        return

    logger.info(f"Loaded evaluation subset of size: {len(eval_dataset)}")
    total_samples = len(eval_dataset)

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'

    prompts = [format_prompt_for_eval(sample) for sample in eval_dataset]

    raw_metrics = {
        "dpo_model": [],
        "sft_model": [],
        "base_model": []
    }

    generation_params = {
        "max_new_tokens": 128,
        "pad_token_id": tokenizer.pad_token_id,
        "batch_size": 4,
        "return_full_text": True,
        "do_sample": False
    }

    # --- Evaluate DPO Model ---
    logger.info("Loading and evaluating DPO model...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        dpo_model = PeftModel.from_pretrained(base_model, dpo_model_path)
        dpo_model = dpo_model.merge_and_unload()

        dpo_pipe = pipeline(
            "text-generation",
            model=dpo_model,
            tokenizer=tokenizer,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )

        dpo_outputs = dpo_pipe(prompts, **generation_params)

        for i, sample in tqdm(enumerate(eval_dataset), desc="Metrics (DPO)", total=total_samples):
            ground_truths = sample['answers']
            available_tools = sample['tools']

            dpo_raw_output = ""
            if dpo_outputs and len(dpo_outputs[i]) > 0:
                dpo_raw_output = dpo_outputs[i][0].get('generated_text', '')

            if not dpo_raw_output:
                logger.warning(f"Skipping DPO sample {i} due to failed/empty generation.")
                continue

            dpo_pred_str = dpo_raw_output.split('<start_of_turn>model\n')[-1].strip()

            dpo_preds = parse_tool_calls(dpo_pred_str)

            ts_f1 = calculate_tool_select_metrics(dpo_preds, ground_truths)
            p, r, f1, em = calculate_metrics(dpo_preds, ground_truths)
            hr = calculate_hallucination_rate(dpo_preds, available_tools)

            raw_metrics["dpo_model"].append({'p': p, 'r': r, 'f1': f1, 'em': em, 'hr': hr, 'ts_f1': ts_f1})

    except Exception as e:
        logger.error(f"Error during DPO Model evaluation: {e}")

    del base_model, dpo_model, dpo_pipe
    gc.collect()
    torch.cuda.empty_cache()
    logger.info("Finished DPO model evaluation and cleared memory.")

    # --- Evaluate SFT Model ---
    logger.info("\nLoading and evaluating SFT model...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        sft_model = PeftModel.from_pretrained(base_model, sft_model_path)
        sft_model = sft_model.merge_and_unload()

        sft_pipe = pipeline(
            "text-generation",
            model=sft_model,
            tokenizer=tokenizer,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )

        sft_outputs = sft_pipe(prompts, **generation_params)

        for i, sample in tqdm(enumerate(eval_dataset), desc="Metrics (SFT)", total=total_samples):
            ground_truths = sample['answers']
            available_tools = sample['tools']

            sft_raw_output = ""
            if sft_outputs and len(sft_outputs[i]) > 0:
                sft_raw_output = sft_outputs[i][0].get('generated_text', '')

            if not sft_raw_output:
                continue

            sft_pred_str = sft_raw_output.split('<start_of_turn>model\n')[-1].strip()

            sft_preds = parse_tool_calls(sft_pred_str)

            ts_f1 = calculate_tool_select_metrics(sft_preds, ground_truths)
            p, r, f1, em = calculate_metrics(sft_preds, ground_truths)
            hr = calculate_hallucination_rate(sft_preds, available_tools)

            raw_metrics["sft_model"].append({'p': p, 'r': r, 'f1': f1, 'em': em, 'hr': hr, 'ts_f1': ts_f1})

    except Exception as e:
        logger.error(f"Error during SFT Model evaluation: {e}")

    del base_model, sft_model, sft_pipe
    gc.collect()
    torch.cuda.empty_cache()
    logger.info("Finished SFT model evaluation and cleared memory.")

    # --- Evaluate Base Model ---
    logger.info("\nLoading and evaluating base model...")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        base_pipe = pipeline(
            "text-generation",
            model=base_model,
            tokenizer=tokenizer,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        base_outputs = base_pipe(prompts, **generation_params)

        logger.setLevel(logging.CRITICAL)

        for i, sample in tqdm(enumerate(eval_dataset), desc="Metrics (Base)", total=total_samples):
            ground_truths = sample['answers']
            available_tools = sample['tools']

            base_raw_output = ""
            if base_outputs and len(base_outputs[i]) > 0:
                base_raw_output = base_outputs[i][0].get('generated_text', '')

            if not base_raw_output:
                continue

            base_pred_str = base_raw_output.split('<start_of_turn>model\n')[-1].strip()

            base_preds = parse_tool_calls(base_pred_str)

            ts_f1 = calculate_tool_select_metrics(base_preds, ground_truths)
            p, r, f1, em = calculate_metrics(base_preds, ground_truths)
            hr = calculate_hallucination_rate(base_preds, available_tools)

            raw_metrics["base_model"].append({'p': p, 'r': r, 'f1': f1, 'em': em, 'hr': hr, 'ts_f1': ts_f1})

        logger.setLevel(logging.INFO)

    except Exception as e:
        logger.error(f"Error during Base Model evaluation: {e}")

    del base_model, base_pipe
    gc.collect()
    torch.cuda.empty_cache()
    logger.info("Finished base model evaluation and cleared memory.")

    # --- SAVE AND PRINT RESULTS ---
    aggregated_results = save_metrics_to_json(raw_metrics, total_samples)

    logger.info("\n" + "="*70)
    logger.info("--- DPO EVALUATION RESULTS SUMMARY ---")
    logger.info("="*70)
    print("\n" + "="*70)
    print("--- DPO EVALUATION RESULTS SUMMARY ---")
    print("="*70)

    for model_name, res_data in aggregated_results.items():

        if "error" in res_data:
            log_message = f"\nModel: {model_name.replace('_', ' ').title()} - {res_data['error']}"
        else:
            log_message = (
                f"\nModel: {model_name.replace('_', ' ').title()} (N={res_data['valid_samples']}/{res_data['total_samples']} samples)\n"
                f"  Exact Match Rate:     {res_data['exact_match_rate']:.4f}\n"
                f"  Tool Call F1:         {res_data['avg_f1_score']:.4f}\n"
                f"  Tool Select F1:       {res_data['avg_tool_select_f1_score']:.4f}\n"
                f"  Precision:            {res_data['avg_precision']:.4f}\n"
                f"  Recall:               {res_data['avg_recall']:.4f}\n"
                f"  Hallucination Rate:   {res_data['avg_hallucination_rate']:.4f}"
            )

        logger.info(log_message)
        print(log_message)

    print("\n" + "="*70)
    print(f"Full results saved to: {RESULTS_FILE}")
    print(f"Log file saved to: {LOG_FILE}")
    print("="*70)

if __name__ == "__main__":
    main()

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Metrics (DPO):  43%|████▎     | 430/1000 [00:00<00:00, 672.56it/s]
ERROR:__main__:Error during DPO Model evaluation: 'int' object is not iterable
Device set to use cuda:0
Metrics (SFT): 100%|██████████| 1000/1000 [00:00<00:00, 5078.85it/s]
Device set to use cuda:0
Metrics (Base): 100%|██████████| 1000/1000 [00:00<00:00, 9131.17it/s]
INFO:__main__:Finished base model evaluation and cleared memory.
INFO:__main__:All evaluation metrics saved to dpo_evaluation_results.json
INFO:__main__:
INFO:__main__:--- DPO EVALUATION RESULTS SUMMARY ---
INFO:__main__:======================================================================
INFO:__main__:
Model: Dpo Model (N=430/1000 samples)
  Exact Match Rate:     0.0000
  Tool Call 


--- DPO EVALUATION RESULTS SUMMARY ---

Model: Dpo Model (N=430/1000 samples)
  Exact Match Rate:     0.0000
  Tool Call F1:         0.0000
  Tool Select F1:       0.0000
  Precision:            0.0000
  Recall:               0.0000
  Hallucination Rate:   0.0233

Model: Sft Model (N=1000/1000 samples)
  Exact Match Rate:     0.2430
  Tool Call F1:         0.2920
  Tool Select F1:       0.8723
  Precision:            0.2933
  Recall:               0.2918
  Hallucination Rate:   0.9820

Model: Base Model (N=1000/1000 samples)
  Exact Match Rate:     0.0000
  Tool Call F1:         0.0000
  Tool Select F1:       0.0000
  Precision:            0.0000
  Recall:               0.0000
  Hallucination Rate:   0.0010

Full results saved to: dpo_evaluation_results.json
Log file saved to: dpo_eval_log_20251126_114953.txt
